In [ ]:
#import necessary libraries
import pandas as pd
from ast import literal_eval
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from itertools import chain
import time


In [ ]:
#load the dataset
pd.set_option('display.max_colwidth', None)
df = pd.read_csv('data/final_dataset.csv', converters={'ProcessTokens': literal_eval})
df.head(1)

In [ ]:
#Converting the datatype of processtokens fromm list to string, so that our vectorization4 works right.
df['ProcessTokens'] = df['ProcessTokens'].astype("string")

In [ ]:
def getSearchTerms():
    genres = ['Action', 'Adventure', 'Casual', 'Free to Play', 'Massively Multiplayer', 'RPG']
    review1 = ["I love plains of eidolon, let's remove every mission in the game and put their rewards in the plains so we can play max range nova every game."]
    review2 = []
    review3 = []
    searchTerms = genres + review1 + review2 + review3
    
    searchTerms = " ".join(map(str, searchTerms))
    searchTerms = searchTerms.lower()
    return searchTerms

In [ ]:
def make_recommendation(df=df):
    new_row = df.iloc[-1,:].copy() #creating a copy of the last row of the 
    #dataset, which we will use to input the user's input

    #grabbing the new wordsoup from the user
    searchTerms = getSearchTerms()
    new_row['soup'] = searchTerms
    new_row['title'] = 'UserInput'

    #adding the new row to the dataset
    df = df.append(new_row)
#     df.iloc[-1] = searchTerms #adding the input to our new row

    #Vectorizing the entire matrix as described above!
    count = CountVectorizer(stop_words='english')
    count_matrix = count.fit_transform(df['soup'])

    #running pairwise cosine similarity 
    cosine_sim2 = cosine_similarity(count_matrix, count_matrix) #getting a similarity matrix

    #sorting cosine similarities by highest to lowest
    sim_scores = list(enumerate(cosine_sim2[-1,:]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    #matching the similarities to the movie titles and ids
    #   ranked_titles = []
    #   for i in range(1, 11):
    #     indx = sim_scores[i][0]
    #     ranked_titles.append([df['title'].iloc[indx], df['soup'].iloc[indx]])
    i = 10
    inc = 1 # we start from the 1-th row because, the 0th row is the input itself. dont want that.
    
    ranked_titles = []
    while i>=0:
        indx = sim_scores[inc][0]
        inc = inc + 1
        current_title = df['title'].iloc[indx]
        if(current_title == 'UserInput'):
            continue
        if(current_title not in chain(*ranked_titles)):
            ranked_titles.append([df['title'].iloc[indx], sim_scores[inc-1][1]])
            i = i - 1

    return ranked_titles

In [ ]:
start_time = time.time()
#let's try our recommendation function now
user_recs = make_recommendation()
display(user_recs)
print("--- %s seconds ---" % (time.time() - start_time))